In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

# TODO: Enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment1/'
FOLDERNAME = 'cs231n/assignments/assignment1/'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# This downloads the CIFAR-10 dataset to your Drive
# if it doesn't already exist.
%cd /content/drive/My\ Drive/$FOLDERNAME/cs231n/datasets/
!bash get_datasets.sh
%cd /content/drive/My\ Drive/$FOLDERNAME

# Softmax 分类器练习

*请完成并提交这份完整的作业工作表（包括输出和任何工作表之外的辅助代码）。更多详情请查看课程网站上的[作业页面](http://vision.stanford.edu/teaching/cs231n/assignments.html)。*

在这个练习中，你将：

- 实现 Softmax 分类器的完全向量化的**损失函数**
- 实现其**解析梯度**的完全向量化表达式
- 使用数值梯度**检查你的实现**
- 使用验证集来**调整学习率和正则化强度**
- 使用**SGD**优化损失函数
- **可视化**最终学到的权重

In [ ]:
# Run some setup code for this notebook.
import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt

# This is a bit of magic to make matplotlib figures appear inline in the
# notebook rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

## CIFAR-10 数据加载和预处理

In [ ]:
# 加载原始 CIFAR-10 数据。
cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'

# 清理变量以防止多次加载数据（可能导致内存问题）
try:
   del X_train, y_train
   del X_test, y_test
   print('清除之前加载的数据。')
except:
   pass

X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

# 作为健全性检查，我们打印出训练和测试数据的大小。
print('训练数据维度: ', X_train.shape)
print('训练标签维度: ', y_train.shape)
print('测试数据维度: ', X_test.shape)
print('测试标签维度: ', y_test.shape)

In [ ]:
# 可视化数据集中的一些示例。
# 我们展示来自每个类别的几个训练图像示例。
classes = ['飞机', '汽车', '鸟', '猫', '鹿', '狗', '青蛙', '马', '船', '卡车']
num_classes = len(classes)
samples_per_class = 7
for y, cls in enumerate(classes):
    idxs = np.flatnonzero(y_train == y)
    idxs = np.random.choice(idxs, samples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt_idx = i * num_classes + y + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        plt.imshow(X_train[idx].astype('uint8'))
        plt.axis('off')
        if i == 0:
            plt.title(cls)
plt.show()

In [ ]:
# 将数据拆分为训练集、验证集和测试集。另外，我们还将
# 创建一个小的开发集作为训练数据的子集；
# 我们可以使用它来开发，这样我们的代码运行得更快。
num_training = 49000
num_validation = 1000
num_test = 1000
num_dev = 500

# 我们的验证集将是从原始
# 训练集中取 num_validation 个点。
mask = range(num_training, num_training + num_validation)
X_val = X_train[mask]
y_val = y_train[mask]

# 我们的训练集将是从原始
# 训练集的前 num_train 个点。
mask = range(num_training)
X_train = X_train[mask]
y_train = y_train[mask]

# 我们还将制作一个开发集，它是
# 训练集的一个小子集。
mask = np.random.choice(num_training, num_dev, replace=False)
X_dev = X_train[mask]
y_dev = y_train[mask]

# 我们使用原始测试集的前 num_test 个点作为我们的
# 测试集。
mask = range(num_test)
X_test = X_test[mask]
y_test = y_test[mask]

print('训练数据维度: ', X_train.shape)
print('训练标签维度: ', y_train.shape)
print('验证数据维度: ', X_val.shape)
print('验证标签维度: ', y_val.shape)
print('测试数据维度: ', X_test.shape)
print('测试标签维度: ', y_test.shape)

In [ ]:
# 预处理：将图像数据重新整形为行
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_val = np.reshape(X_val, (X_val.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))
X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))

# 作为健全性检查，打印数据的维度
print('训练数据维度: ', X_train.shape)
print('验证数据维度: ', X_val.shape)
print('测试数据维度: ', X_test.shape)
print('开发数据维度: ', X_dev.shape)

In [ ]:
# 预处理：减去平均图像
# 首先：基于训练数据计算图像均值
mean_image = np.mean(X_train, axis=0)
print(mean_image[:10]) # 打印几个元素
plt.figure(figsize=(4,4))
plt.imshow(mean_image.reshape((32,32,3)).astype('uint8')) # 可视化平均图像
plt.show()

# 第二：从训练和测试数据中减去平均图像
X_train -= mean_image
X_val -= mean_image
X_test -= mean_image
X_dev -= mean_image

# 第三：添加偏置维度（偏置技巧）以便我们的分类器
# 只需要担心优化一个权重矩阵 W。
X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])

print(X_train.shape, X_val.shape, X_test.shape, X_dev.shape)

## Softmax 分类器

你为本节编写的代码都将写在 `cs231n/classifiers/softmax.py` 中。

如你所见，我们已经预填充了 `softmax_loss_naive` 函数，它使用 for 循环来评估 softmax 损失函数。

In [ ]:
# 评估我们为你提供的朴素实现：
from cs231n.classifiers.softmax import softmax_loss_naive
import time

# 生成一个由小数组成的随机 Softmax 分类器权重矩阵
W = np.random.randn(3073, 10) * 0.0001

loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.000005)
print('损失: %f' % (loss, ))

# 作为一个粗略的健全性检查，我们的损失应该接近 -log(0.1)。
print('损失: %f' % loss)
print('健全性检查: %f' % (-np.log(0.1)))

**内联问题 1**

为什么我们期望损失接近 -log(0.1)？简要解释。

$\color{blue}{\textit 你的答案：}$ *在此填写*

上面函数返回的 `grad` 现在全是零。推导并实现 softmax 损失函数的梯度，并在 `softmax_loss_naive` 函数内部内联实现它。你会发现将新代码交织在现有函数中很有帮助。

为了检查你正确实现了梯度，你可以数值估计损失函数的梯度，并与你计算的梯度进行比较。我们为你提供了执行此操作的代码：

In [ ]:
# 一旦你实现了梯度，使用下面的代码重新计算它
# 并使用我们提供的函数进行梯度检查

# 计算在 W 处的损失及其梯度。
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# 在几个随机选择的维度上数值计算梯度，并
# 将它们与你解析计算的梯度进行比较。数字应该在
# 所有维度上几乎完全匹配。
from cs231n.gradient_check import grad_check_sparse
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, grad)

# 在开启正则化的情况下再次进行梯度检查
# 你没有忘记正则化梯度吧？
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 5e1)
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 5e1)[0]
grad_numerical = grad_check_sparse(f, W, grad)

**内联问题 2**

虽然 gradcheck 对于可靠的 softmax 损失，但有时对于 SVM 损失，gradcheck 中偶尔会有一个维度不完全匹配。造成这种差异的原因是什么？值得担心吗？在什么情况下 SVM 损失梯度检查可能失败的简单例子是什么？改变边距如何影响这种情况发生的频率？

请注意，对于样本 $(x_i, y_i)$，SVM 损失定义为：$$L_i = \sum_{j\ne y_i}\max(0, s_j - s_{y_i} + \Delta)$$ 其中 $j$ 遍历除正确类别 $y_i$ 外的所有类别，$s_j$ 表示第 $j$ 个类别的分类器分数。$\Delta$ 是一个标量边距。有关更多信息，请参阅[此](https://cs231n.github.io/linear-classify/)页面上的"多类支持向量机损失"。

*提示：SVM 损失函数严格来说不是可微的。*

$\color{blue}{\textit 你的答案：}$ *在此填写*。

In [ ]:
# 接下来实现函数 softmax_loss_vectorized；现在只计算损失；
# 我们稍后会实现梯度。
tic = time.time()
loss_naive, grad_naive = softmax_loss_naive(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('朴素损失: %e 用时 %fs' % (loss_naive, toc - tic))

from cs231n.classifiers.softmax import softmax_loss_vectorized
tic = time.time()
loss_vectorized, _ = softmax_loss_vectorized(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('向量化损失: %e 用时 %fs' % (loss_vectorized, toc - tic))

# 损失应该匹配，但你的向量化实现应该快得多。
print('差异: %f' % (loss_naive - loss_vectorized))

In [ ]:
# 完成 softmax_loss_vectorized 的实现，并以向量化的方式计算损失函数的梯度。

# 朴素实现和向量化实现应该匹配，但
# 向量化版本仍然应该快得多。
tic = time.time()
_, grad_naive = softmax_loss_naive(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('朴素损失和梯度: 用时 %fs' % (toc - tic))

tic = time.time()
_, grad_vectorized = softmax_loss_vectorized(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('向量化损失和梯度: 用时 %fs' % (toc - tic))

# 损失是一个数字，所以很容易比较
# 两种实现计算的值。另一方面，梯度是一个矩阵，所以
# 我们使用 Frobenius 范数来比较它们。
difference = np.linalg.norm(grad_naive - grad_vectorized, ord='fro')
print('差异: %f' % difference)

### 随机梯度下降

现在我们有了向量化且高效的损失表达式、梯度，并且我们的梯度与数值梯度匹配。因此我们准备进行 SGD 以最小化损失。你为本节编写的代码将写在 `cs231n/classifiers/linear_classifier.py` 中。

In [ ]:
# 在文件 linear_classifier.py 中，在函数
# LinearClassifier.train() 中实现 SGD，然后运行下面的代码。
from cs231n.classifiers import Softmax
softmax = Softmax()
tic = time.time()
loss_hist = softmax.train(X_train, y_train, learning_rate=1e-7, reg=2.5e4,
                      num_iters=1500, verbose=True)
toc = time.time()
print('用时 %fs' % (toc - tic))

In [ ]:
# 一个有用的调试策略是将损失绘制为
# 迭代次数的函数：
plt.plot(loss_hist)
plt.xlabel('迭代次数')
plt.ylabel('损失值')
plt.show()

In [ ]:
# 编写 LinearClassifier.predict 函数并在
# 训练集和验证集上评估性能
# 你应该得到大约 0.34 的验证准确率（> 0.33）。
y_train_pred = softmax.predict(X_train)
print('训练准确率: %f' % (np.mean(y_train == y_train_pred), ))
y_val_pred = softmax.predict(X_val)
print('验证准确率: %f' % (np.mean(y_val == y_val_pred), ))

In [ ]:
# 保存训练好的模型用于自动评分。
softmax.save("softmax.npy")

In [ ]:
# 使用验证集来调整超参数（正则化强度和学习率）。
# 你应该尝试不同的学习率和正则化强度范围；如果你仔细调整，
# 你应该能在验证集上获得大约 0.365（> 0.36）的分类准确率。

# 注意：在超参数搜索期间，你可能会看到运行时/溢出警告。
# 这可能是由极端值引起的，并不是错误。

# results 是一个字典，将形式为
# (learning_rate, regularization_strength) 的元组映射到
# 形式为 (training_accuracy, validation_accuracy) 的元组。准确率简单地
# 是被正确分类的数据点的分数。
results = {}
best_val = -1   # 到目前为止我们看到的最高验证准确率。
best_softmax = None # 达到最高验证率的 Softmax 对象。

################################################################################
# TODO:                                                                        #
# 编写在验证集上调整超参数的代码。对于每组超参数组合，训练一个 Softmax，#
# 在训练集上训练，在训练集和验证集上计算其准确率，并将这些数字存储在     #
# results 字典中。此外，将最佳验证准确率存储在 best_val 中，实现这一点的   #
# Softmax 对象存储在 best_softmax 中。                                        #
#                                                                              #
# 提示：在开发验证代码时，你应该使用较小的 num_iters 值，这样分类器就不会花很多时间训练；一旦你确信验证代码有效，你应该用较大的 num_iters 值重新运行代码。 #
################################################################################

# 作为参考提供。你可能想也可能不想改变这些超参数
learning_rates = [1e-7, 1e-6]
regularization_strengths = [2.5e4, 1e4]



# 打印结果。
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e 训练准确率: %f 验证准确率: %f' % (
                lr, reg, train_accuracy, val_accuracy))

print('交叉验证期间达到的最佳验证准确率: %f' % best_val)

In [ ]:
# 可视化交叉验证结果
import math
import pdb

# pdb.set_trace()

x_scatter = [math.log10(x[0]) for x in results]
y_scatter = [math.log10(x[1]) for x in results]

# 绘制训练准确率
marker_size = 100
colors = [results[x][0] for x in results]
plt.subplot(2, 1, 1)
plt.tight_layout(pad=3)
plt.scatter(x_scatter, y_scatter, marker_size, c=colors, cmap=plt.cm.coolwarm)
plt.colorbar()
plt.xlabel('log 学习率')
plt.ylabel('log 正则化强度')
plt.title('CIFAR-10 训练准确率')

# 绘制验证准确率
colors = [results[x][1] for x in results] # 默认标记大小为 20
plt.subplot(2, 1, 2)
plt.scatter(x_scatter, y_scatter, marker_size, c=colors, cmap=plt.cm.coolwarm)
plt.colorbar()
plt.xlabel('log 学习率')
plt.ylabel('log 正则化强度')
plt.title('CIFAR-10 验证准确率')
plt.show()

In [ ]:
# 在测试集上评估最佳 softmax
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('在原始像素上的 Softmax 分类器最终测试集准确率: %f' % test_accuracy)

In [ ]:
# 保存最佳 softmax 模型
best_softmax.save("best_softmax.npy")

In [ ]:
# 可视化每个类别学到的权重。
# 根据你选择的学习率和正则化强度，这些可能
# 看起来不错也可能不怎么样。
w = best_softmax.W[:-1,:] # 去掉偏置
w = w.reshape(32, 32, 3, 10)
w_min, w_max = np.min(w), np.max(w)
classes = ['飞机', '汽车', '鸟', '猫', '鹿', '狗', '青蛙', '马', '船', '卡车']
for i in range(10):
    plt.subplot(2, 5, i + 1)

    # 将权重重新缩放到 0 到 255 之间
    wimg = 255.0 * (w[:, :, :, i].squeeze() - w_min) / (w_max - w_min)
    plt.imshow(wimg.astype('uint8'))
    plt.axis('off')
    plt.title(classes[i])

**内联问题 3**

描述你可视化的 Softmax 分类器权重是什么样的，并简要解释为什么它们看起来是这样的。

$\color{blue}{\textit 你的答案：}$ *在此填写*

**内联问题 4** - *正确或错误*

假设总体训练损失定义为所有训练示例上逐数据点损失的总和。可以向训练集添加一个新的数据点，该数据点会改变 softmax 损失，但使 SVM 损失保持不变。

$\color{blue}{\textit 你的答案：}$


$\color{blue}{\textit 你的解释：}$